In [1]:
import keras
from keras import models, layers
from keras import backend

Using TensorFlow backend.


In [2]:
class CNN(models.Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        self.add(layers.Conv2D(32, kernel_size=(3, 3), 
                               activation='relu', 
                               input_shape=input_shape))
        
        self.add(layers.Conv2D(64, (3, 3), activation='relu'))
        self.add(layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(layers.Dropout(0.25))
        self.add(layers.Flatten())
        
        self.add(layers.Dense(128, activation='relu'))
        self.add(layers.Dropout(0.5))
        
        self.add(layers.Dense(num_classes, activation='softmax'))
        
        self.compile(loss=keras.losses.categorical_crossentropy, 
                     optimizer='rmsprop',
                     metrics=['accuracy'])

In [3]:
from keras import datasets

class DATA():
    def __init__(self):
        num_classes = 10
        
        (X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
        
        img_rows, img_cols = X_train.shape[1:]
        
        if backend.image_data_format() == "channels_first":
            X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
            X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, img_rows, img_cols)
        else: 
            X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
            X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
            input_shape = (img_rows, img_cols, 1)
            
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255
        
        Y_train = keras.utils.to_categorical(y_train, num_classes)
        Y_test = keras.utils.to_categorical(y_test, num_classes)
        
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.X_train, self.Y_train = X_train, Y_train
        self.X_test, self.Y_test = X_test, Y_test

In [4]:
from keraspp.skeras import plot_acc, plot_loss
import matplotlib.pyplot as plt

In [5]:
def main():
    batch_size = 128
    epochs = 10
    
    data = DATA()
    model = CNN(data.input_shape, data.num_classes)
    
    history = model.fit(data.X_train, data.Y_train, 
                        batch_size=batch_size, 
                        epochs=epochs, 
                        validation_split=0.2)
    
    score = model.evaluate(data.X_test, data.Y_test)
    print()
    print("Test Loss: ", score[0])
    print("Test Accuracy:", score[1])
    
    plot_acc(history)
    plt.show()
    plot_loss(history)
    plt.show()

In [6]:
if __name__=='__main__':
    main()

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
44928/48000 [===========================>..] - ETA: 7s - loss: 0.2628 - acc: 0.9182

KeyboardInterrupt: 